# Understanding the Dataset 1
## 1. Import the data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
col_index = [1,2,3,4,6,7,9,10,12,13,15,16]
col_names = ['re_time','re_ID','re_x','re_y','tr_time','tr_ID','tr_x','tr_y','tr_vx','tr_vy','RSSI','Label']
data = pd.read_csv('dataset/attack1withlabels.csv', usecols = col_index, header = None, names = col_names)
data_atk1 = data.dropna(axis = 0, how = 'any')

## 2.1. The line-of-signt range 
We assume the LoS is 800m, and check

In [ ]:
def distance(X):
    distance = np.linalg.norm([X['re_x']-X['tr_x'], X['re_y']-X['tr_y']])
    return distance

def filtering_dis(data_atk1):
    
    length = data_atk1.shape[0]
    dist = np.zeros(length)
    for i in range(length):
        a = data_atk1.iloc[i]
        m = distance(a) 
        dist[i] = m
    distance_series = pd.Series(dist)
    return distance_series

def add_distance(data_atk1, distance_series):
    data_atk1['Distance'] = distance_series
    return data_atk1

def row_out_range(data_atk1, thredhold):
    length = data_atk1.shape[0]
    drop_index = np.zeros(length)
    for i in range(length):
        if data_atk1['Distance'][i] > thredhold:
            drop_index[i]=1  
    drop_index_ = np.where(drop_index>0)
    drop_row = np.asarray(drop_index_)[0]
    return drop_row

def drop_distance(data_atk1, thredhold):
    drop_row = row_out_range(data_atk1, thredhold)
    data_drop = data_atk1.drop(drop_row)
    return data_drop

In [ ]:
distance_series = filtering_dis(data_atk1)
data_atk1_index = data_atk1.reset_index(drop = True)
data_atk1_index['Distance'] = distance_series

In [ ]:
row_drop_800 = row_out_range(data_atk1_index,800)

In [ ]:
plus_800 = data_atk1_index.iloc[row_drop_800]
label_plus_800 = np.unique(np.array(plus_800.iloc[:,11]))
print('All the BSM with the distance over than 800m have the label', label_plus_800,', They are all labelled as attack')

## 2.2. Check the data for some insights
I want to show you that each receiver is whether attack or not. 
Firstly, 

In [ ]:
# We filter the data and re index the data. 
data_less800 = drop_distance(data_atk1_index,800)
data_less800 = data_less800.reset_index(drop = True)
recevier_ID = np.unique(np.array(data_atk1.iloc[:,1]))
sender_ID = np.unique(np.array(data_atk1.iloc[:,5]))

In [ ]:
def _check(data):
    count = 0
    for i in range(len(recevier_ID)):
        data_recevier = data[data['re_ID'] == recevier_ID[i]]
        sender_receiver = np.unique(np.array(data_recevier.iloc[:,5]))
        count += len(sender_receiver)
    print('There are', count, 'times communication between all sender and all receiver')

def duration_check(data):
    duration = []
    label = []
    for i in range(len(recevier_ID)):
        data_recevier = data[data['re_ID'] == recevier_ID[i]]
        sender_receiver = np.unique(np.array(data_recevier.iloc[:,5]))
        for j in range(len(sender_receiver)):
            dura = data_recevier[data_recevier['tr_ID'] == sender_receiver[j]]
            label_this = np.unique(np.array(dura.iloc[:,11]))
            duration.append(dura.shape[0])   
            label.append(label_this)
    return duration, label  

In [ ]:
_check(data_atk1)

In [ ]:
# a: is the duration of each session of communication
# b: is 'whether the sender during this session is attack or not'
a, b = duration_check(data_less800)

Here we noticed for each transmission, it is whether normal or abnormal. 

In [ ]:
def sender_check(data_less800):
    length = len(sender_ID)
    label = np.zeros(length)
    for i in range(length):
        dura = data_less800[data_less800['tr_ID'] == sender_ID[i]]
        label_this = np.unique(np.array(dura.iloc[:,11]))
        label[i] = label_this
    return label

def identify_sender(data_less800):
    length = len(sender_ID)
    label = []
    for i in range(length):
        dura = data_less800[data_less800['tr_ID'] == sender_ID[i]]
        label_this = np.unique(np.array(dura.iloc[:,11]))
        if label_this==1:
            print('The messages from the sender', sender_ID[i], 'are malicious')
            label.append(sender_ID[i])
    return label

For each senders, we check all the messages from him: is it all 'attack', or all 'normal', or mix of 'attack' and 'normal'

In [ ]:
m = sender_check(data_less800)
print(m)

After I apply the function unique, the return is [0] or [1], there is no [0,1].It means for all the sender, it is whether attack or normal, no mix

In [ ]:
# And then we find out the malicious sender. 
sender_ID_malicious = identify_sender(data_less800)

## 2.3. Here I show a typical attack, and get its insight. 

In [ ]:
def display_the_data(data_each_communication):
    fig, ax1 = plt.subplots()
    t = np.array(data_each_communication['re_time'])
    dis = np.array(data_each_communication['Distance'])
    RSSI_of_it = np.array(data_each_communication['RSSI'])
    
    color = 'tab:blue'
    ax1.set_xlabel('The time series')
    ax1.set_ylabel('distance', color=color)
    ax1.plot(t, dis, 'b-')
    ax1.tick_params(axis='y', labelcolor=color)
    
    ax2 = ax1.twinx()
    color = 'tab:red'
    ax2.set_ylabel('RSSI', color=color)  # we already handled the x-label with ax1
    ax2.plot(t, RSSI_of_it, 'r--')
    ax2.tick_params(axis='y', labelcolor=color)
    
    fig.tight_layout() 
    plt.show()

In [ ]:
dura = data_less800[data_less800['tr_ID'] == sender_ID_malicious[2]]
receiver_malicious = np.unique(np.array(dura.iloc[:,1]))
data_each_communication = dura[dura['re_ID'] == receiver_malicious[1]]
data_each_communication # This is a malicous data for one transmition

In [ ]:
display_the_data(data_each_communication)

In [ ]:
def present_malicious(sender_ID_malicious, data_less800):
    length = len(sender_ID_malicious)
    count = 0
    for i in range(length):
        dura = data_less800[data_less800['tr_ID'] == sender_ID_malicious[i]]
        receiver_malicious = np.unique(np.array(dura.iloc[:,1]))
        number_receiver = len(receiver_malicious)
        for j in range(number_receiver):
            data_each_communication = dura[dura['re_ID'] == receiver_malicious[j]]
            ### We might add more data analysis here. 
            display_the_data(data_each_communication)
            count = count + 1
            if count > 10:
                return
    return


In [ ]:
present_malicious(sender_ID_malicious, data_less800)

We observe that both the position and the distance does not change rapidly but the RSSI changes rapidly. 

## 3. Conclusions from the dataset
From the experiment we can get some insight from the dataset:
* Each receiver a series of data from some sender, the receiver utilize the data to detect malicious behaviour. Thus, it is important to extract a session of communication. 
* Each sender is whether malicious or not, there is no sender who send some normal BSM message and some abnornal message. Thus, we could utilize the time series to train the neural network. 
* I suggest that we reconstruct the data set into small data-patch of duration of 10 \* 0.1 ms. 

## Limits
* We have to admit that for some transmission session, there is only one or two BSM package, thus it is impossible to comstruct a time series, so we have to figure out a method to overlook this data. 

## Get the clean data
Logically, for each receiver who checks the incoming message is malicious or not, a series of BSM from one sender is logged. Thus the input information to the network could be the data in a duration, for example, 0.5s or 1s. We thus slice the data into small sections (we start with 5)

In [33]:
time_series_length = 5

def number_of_communication(data, recevier_ID_all, sender_ID_all):
    number_recevier_ID_all = len(recevier_ID_all)
    count = 0
    for i in range(number_recevier_ID_all):
        data_this_recevier = data[data['re_ID'] == recevier_ID_all[i]]
        sender_receiver = np.unique(np.array(data_this_recevier.iloc[:,5]))
        count = count + len(sender_receiver)
    return count

def clean_session(session, time_series_length):
    
    length = session.shape[0]
    t = time_series_length
    m = 7 # m is the useful infomation from each BSM
    
    
    info = m * t + 1  
    remained = length%t
    if remained == 0:
        len_info = length//t
    else:
        len_info = length//t + 1
        
    session_vector = np.zeros((len_info, info))
    
    if remained > 0:
        for h in range(t-remained):
            session = session.append(session.iloc[-1], ignore_index=True)
            
    for i in range(len_info):
        session_section = session.iloc[[i*t, i*t+1, i*t+2, i*t+3, i*t+4]]
        session_vector[i, -1] = session_section.iloc[0][-1]
        for j in range(t):
            session_vector[i, m*j] = abs(session_section.iloc[j][2] - session_section.iloc[j][6]) 
            session_vector[i, m*j+1] = abs(session_section.iloc[j][3] - session_section.iloc[j][7])
            session_vector[i, m*j+2] = session_section.iloc[j][6]
            session_vector[i, m*j+3] = session_section.iloc[j][7]
            session_vector[i, m*j+4] = session_section.iloc[j][8]
            session_vector[i, m*j+5] = session_section.iloc[j][9]
            session_vector[i, m*j+6] = session_section.iloc[j][10]
    return session_vector
            
            
            
            
def knife(data, time_series_length):
    recevier_ID_all = np.unique(np.array(data.iloc[:,1]))
    sender_ID_all = np.unique(np.array(data.iloc[:,5]))
    
    a = number_of_communication(data, recevier_ID_all, sender_ID_all)
    print('There are', a, 'sessions of communication in total')
        
    for i in range(len(recevier_ID_all)):
        data_recevier = data[data['re_ID'] == recevier_ID_all[i]]
        sender_receiver = np.unique(np.array(data_recevier.iloc[:,5]))
        for j in range(len(sender_receiver)):
            this_session = data_recevier[data_recevier['tr_ID'] == sender_receiver[j]]
            # We clean the session and append it to original data 
            
            slice_session = clean_session(this_session, time_series_length)
            print(recevier_ID_all[i], sender_receiver[j])
            
            if i==0 and j==0:
                clean_data = slice_session
            else:
                clean_data = np.append(clean_data, slice_session, axis=0)
    return clean_data


        

In [20]:
data_recevier = data[data['re_ID'] == 10]
#sender_receiver = np.unique(np.array(data_recevier.iloc[:,5]))
session = data_recevier[data_recevier['tr_ID'] == 19]

In [21]:
#session.iloc[[0, 1, 2, 3, 4]]
session

,re_time,re_ID,re_x,re_y,tr_time,tr_ID,tr_x,tr_y,tr_vx,tr_vy,RSSI,Label
382284,21601,10,3584.4,5817.8,21601,19,3623.8,5322.3,-3.5671,29.729,4.460000e-09,0
382301,21602,10,3584.3,5849.3,21602,19,3620.3,5352.1,-3.4325,29.742,4.530000e-09,0
382355,21606,10,3589.6,5974.6,21606,19,3606.6,5470.9,-3.4314,29.733,1.660000e-09,0
382365,21607,10,3591.9,6005.7,21607,19,3603.9,5500.7,-2.4403,29.790,1.370000e-09,0
382376,21608,10,3595.1,6036.8,21608,19,3601.4,5530.5,-2.4397,29.782,3.000000e-09,0


In [34]:
data_for_network = knife(data_atk1, 5)

There are 30588 sessions of communication in total
10 13
10 19
10 25
10 31
10 37
10 43
10 49
10 55
10 61
10 67
10 79
10 91
10 97
10 145
10 277
10 319
10 331
10 337
10 385
10 457
10 475
10 553
10 571
10 595
10 619
10 643
16 7
16 19
16 25
16 31
16 37
16 43
16 49
16 55
16 61
16 67
16 79
16 91
16 97
16 277
16 319
16 331
16 337
16 385
16 457
16 475
16 553
16 571
16 595
22 7
22 13
22 25
22 31
22 37
22 43
22 49
22 55
22 61
22 67
22 79
22 91
22 97
22 145
22 169
22 259
22 265
22 277
22 289
22 319
22 331
22 337
22 349
22 391
22 457
22 475
22 523
22 553
22 565
22 571
22 595
22 619
22 643
22 661
22 679
22 685
22 691
22 697
22 703
22 715
22 721
22 733
22 751
22 787
22 811
28 7
28 13
28 19
28 37
28 43
28 49
28 55
28 61
28 67
28 79
28 91
28 97
28 145
28 169
28 259
28 265
28 277
28 289
28 319
28 331
28 337
28 349
28 385
28 391
28 451
28 457
28 475
28 523
28 529
28 553
28 565
28 571
28 595
28 619
28 643
28 661
28 679
28 685
28 691
28 697
28 703
28 715
28 721
28 733
28 757
28 775
28 781
28 793
28 799
34

172 1987
172 1993
172 2005
172 2017
178 121
178 151
178 307
178 355
178 361
178 421
178 577
184 139
184 247
184 295
184 439
184 535
190 121
190 133
190 157
190 193
190 211
190 217
190 223
190 229
190 235
190 241
190 295
190 301
190 307
190 313
190 343
190 355
190 379
190 403
190 409
190 433
190 487
190 493
190 505
190 535
190 589
190 601
190 613
190 631
190 667
190 709
190 769
190 805
190 817
190 823
190 835
190 877
190 961
196 73
196 115
196 121
196 133
196 139
196 157
196 187
196 211
196 217
196 223
196 229
196 235
196 241
196 295
196 301
196 307
196 313
196 343
196 379
196 397
196 403
196 409
196 439
196 487
196 493
196 505
196 535
196 589
196 601
196 607
196 613
196 625
196 631
196 649
196 655
196 667
196 673
196 709
196 727
196 763
196 769
196 805
196 817
196 823
196 835
196 877
196 961
196 1057
196 1201
202 115
202 217
202 247
202 301
202 439
202 493
208 85
208 103
208 163
208 169
208 259
208 265
208 283
208 325
208 349
208 451
208 463
208 511
208 523
208 529
208 571
208 685
208 

286 3313
286 3319
286 3325
286 3337
286 3343
286 3355
286 3361
286 3379
286 3385
286 3391
286 3403
286 3409
286 3415
286 3427
286 3439
286 3445
286 3451
286 3463
286 3469
286 3475
286 3481
286 3487
286 3499
286 3523
286 3577
286 3619
286 3703
292 19
292 25
292 31
292 37
292 43
292 55
292 61
292 67
292 79
292 85
292 91
292 145
292 163
292 169
292 259
292 265
292 271
292 277
292 319
292 331
292 337
292 349
292 391
292 415
292 445
292 451
292 457
292 469
292 475
292 481
292 499
292 511
292 523
292 529
292 565
292 571
292 595
292 643
292 661
292 679
292 685
292 691
292 697
292 703
292 715
292 721
292 757
292 775
292 781
292 793
292 853
292 871
292 913
292 1321
292 1393
298 115
298 121
298 133
298 139
298 157
298 181
298 187
298 193
298 211
298 217
298 223
298 229
298 235
298 241
298 247
298 301
298 307
298 313
298 343
298 379
298 397
298 403
298 409
298 439
298 487
298 493
298 505
298 535
298 541
298 589
298 601
298 607
298 613
298 625
298 631
298 637
298 649
298 655
298 667
298 673
298 70

394 1981
394 2023
394 2047
394 2077
394 2179
394 2269
394 2299
394 2329
394 2347
394 2425
394 2497
394 2851
400 133
400 139
400 193
400 211
400 235
400 295
400 301
400 343
400 403
400 409
400 427
400 487
400 493
400 505
400 535
400 589
406 121
406 133
406 139
406 157
406 187
406 193
406 211
406 217
406 223
406 229
406 235
406 241
406 295
406 301
406 307
406 313
406 343
406 379
406 397
406 409
406 487
406 493
406 505
406 535
406 589
406 601
406 607
406 613
406 631
406 667
406 709
406 763
406 769
406 805
406 817
406 823
406 835
406 859
406 877
406 889
406 943
406 961
406 1057
406 1201
406 1225
406 1243
406 1255
406 1273
406 1291
406 1297
412 121
412 133
412 139
412 157
412 187
412 193
412 211
412 217
412 223
412 229
412 235
412 295
412 301
412 307
412 343
412 379
412 397
412 403
412 487
412 505
412 535
412 589
412 607
412 613
412 631
412 667
412 709
412 763
412 769
412 805
412 817
412 823
412 835
412 877
412 943
412 961
412 1057
412 1201
412 1225
412 1243
412 1255
412 1273
412 1291
418 7

532 1075
532 1081
532 1099
532 1123
532 1135
532 1141
532 1147
532 1153
532 1159
532 1165
532 1171
532 1177
532 1189
532 1195
532 1207
532 1219
532 1237
532 1261
532 1267
532 1285
532 1315
532 1441
538 115
538 121
538 133
538 139
538 157
538 181
538 187
538 193
538 211
538 217
538 223
538 229
538 235
538 241
538 247
538 295
538 301
538 307
538 313
538 343
538 379
538 397
538 403
538 409
538 439
538 487
538 493
538 505
538 541
538 589
538 601
538 607
538 613
538 625
538 631
538 637
538 649
538 655
538 667
538 673
538 709
538 727
538 763
538 769
538 805
538 817
538 823
538 835
538 877
538 961
538 1057
538 1201
538 1225
538 1243
538 1255
544 115
544 139
544 211
544 235
544 247
544 295
544 301
544 313
544 343
544 439
544 487
544 601
544 625
544 631
544 637
544 649
544 655
544 667
544 673
544 727
550 121
550 253
550 271
550 307
550 355
550 433
550 559
550 577
556 7
556 13
556 19
556 25
556 43
556 49
556 55
556 61
556 67
556 79
556 91
556 97
556 145
556 163
556 169
556 265
556 277
556 289
55

...

3256 1093
3256 1117
3256 1183
3256 1393
3256 1405
3256 1597
3256 1621
3256 1729
3256 1951
3256 1987
3256 1993
3256 2119
3256 2269
3256 2473
3256 2557
3256 2671
3256 2761
3256 2779
3256 2791
3256 2851
3256 2929
3256 2935
3256 3055
3256 3067
3256 3079
3256 3085
3256 3097
3256 3103
3256 3109
3256 3115
3256 3121
3256 3127
3256 3139
3256 3157
3256 3163
3256 3169
3256 3187
3256 3193
3256 3205
3256 3217
3256 3223
3256 3235
3256 3241
3256 3259
3256 3271
3256 3277
3256 3313
3256 3319
3256 3325
3256 3337
3256 3343
3256 3361
3256 3379
3256 3385
3256 3391
3256 3403
3256 3409
3256 3427
3256 3451
3256 3475
3256 3481
3256 3499
3256 3517
3256 3523
3262 283
3262 1087
3262 1093
3262 1117
3262 1183
3262 1393
3262 1405
3262 1597
3262 1729
3262 1951
3262 1987
3262 1993
3262 2119
3262 2473
3262 2557
3262 2671
3262 2761
3262 2779
3262 2791
3262 2929
3262 2983
3262 3031
3262 3043
3262 3049
3262 3067
3262 3073
3262 3079
3262 3085
3262 3091
3262 3103
3262 3109
3262 3115
3262 3121
3262 3127
3262 3133
3262 3139
3

3346 3499
3346 3511
3346 3535
3346 3547
3346 3553
3346 3577
3346 3589
3346 3595
3346 3601
3346 3607
3346 3613
3346 3631
3346 3643
3352 925
3352 1843
3352 2515
3352 2533
3352 2821
3352 2893
3352 2923
3352 2995
3352 3001
3352 3007
3352 3013
3352 3025
3352 3037
3352 3211
3352 3265
3352 3283
3352 3331
3352 3367
3352 3397
3352 3457
3352 3493
3352 3541
3352 3559
3352 3571
3352 3583
3352 3655
3352 3661
3352 3673
3352 3691
3352 3721
3358 283
3358 1093
3358 1117
3358 1393
3358 1405
3358 1621
3358 1951
3358 2119
3358 2473
3358 2671
3358 2761
3358 2779
3358 2791
3358 3187
3358 3193
3358 3205
3358 3217
3358 3259
3358 3277
3358 3295
3358 3307
3358 3313
3358 3319
3358 3325
3358 3337
3358 3343
3358 3361
3358 3373
3358 3379
3358 3385
3358 3391
3358 3403
3358 3409
3358 3415
3358 3421
3358 3427
3358 3439
3358 3445
3358 3451
3358 3463
3358 3469
3358 3475
3358 3481
3358 3487
3358 3499
3358 3511
3358 3535
3358 3547
3358 3553
3358 3577
3358 3589
3358 3601
3358 3607
3358 3613
3358 3619
3358 3625
3358 3637
33

3442 3307
3442 3313
3442 3319
3442 3325
3442 3337
3442 3343
3442 3355
3442 3361
3442 3373
3442 3379
3442 3385
3442 3391
3442 3403
3442 3409
3442 3415
3442 3421
3442 3427
3442 3445
3442 3451
3442 3463
3442 3469
3442 3475
3442 3481
3442 3487
3442 3499
3442 3511
3442 3535
3442 3547
3442 3553
3442 3577
3442 3589
3442 3595
3442 3601
3442 3607
3442 3613
3442 3619
3442 3625
3442 3631
3442 3637
3442 3649
3442 3679
3442 3685
3442 3697
3448 1087
3448 1117
3448 1393
3448 1405
3448 1621
3448 1951
3448 1987
3448 2119
3448 2473
3448 2671
3448 2761
3448 2779
3448 2791
3448 3067
3448 3193
3448 3217
3448 3277
3448 3295
3448 3307
3448 3313
3448 3319
3448 3325
3448 3337
3448 3343
3448 3355
3448 3361
3448 3373
3448 3379
3448 3385
3448 3391
3448 3403
3448 3409
3448 3415
3448 3421
3448 3427
3448 3439
3448 3451
3448 3463
3448 3469
3448 3475
3448 3481
3448 3487
3448 3499
3448 3511
3448 3535
3448 3547
3448 3553
3448 3577
3448 3589
3448 3595
3448 3601
3448 3607
3448 3613
3448 3619
3448 3625
3448 3637
3448 3649


3556 3409
3556 3415
3556 3421
3556 3427
3556 3439
3556 3445
3556 3451
3556 3463
3556 3469
3556 3475
3556 3481
3556 3487
3556 3499
3556 3511
3556 3535
3556 3547
3556 3577
3556 3589
3556 3595
3556 3601
3556 3607
3556 3613
3556 3619
3556 3625
3556 3631
3556 3637
3556 3643
3556 3649
3556 3679
3556 3685
3556 3697
3556 3703
3556 3727
3562 2515
3562 2533
3562 2923
3562 2995
3562 3001
3562 3025
3562 3211
3562 3283
3562 3331
3562 3349
3562 3367
3562 3397
3562 3457
3562 3493
3562 3541
3562 3571
3562 3583
3562 3667
3562 3715
3562 3733
3568 2479
3568 2485
3568 2569
3568 2653
3568 2677
3568 2827
3568 2935
3568 3007
3568 3529
3574 2515
3574 2533
3574 2893
3574 2995
3574 3001
3574 3025
3574 3037
3574 3331
3574 3349
3574 3367
3574 3397
3574 3457
3574 3493
3574 3541
3574 3559
3580 283
3580 1117
3580 1183
3580 1393
3580 1405
3580 1729
3580 1951
3580 1987
3580 2119
3580 2473
3580 2671
3580 2761
3580 2791
3580 3067
3580 3193
3580 3319
3580 3325
3580 3337
3580 3343
3580 3355
3580 3361
3580 3373
3580 3379
3

3688 3703
3694 2515
3694 3349
3694 3397
3694 3457
3694 3583
3694 3655
3694 3661
3694 3667
3694 3721
3700 283
3700 1117
3700 1393
3700 1621
3700 1729
3700 2119
3700 2473
3700 2671
3700 2761
3700 2791
3700 3439
3700 3445
3700 3463
3700 3487
3700 3511
3700 3535
3700 3547
3700 3553
3700 3577
3700 3589
3700 3595
3700 3601
3700 3607
3700 3613
3700 3619
3700 3625
3700 3631
3700 3637
3700 3643
3700 3649
3700 3679
3700 3685
3700 3703
3706 1117
3706 1393
3706 1621
3706 2119
3706 2269
3706 2473
3706 2671
3706 2761
3706 2779
3706 2791
3706 3067
3706 3439
3706 3445
3706 3487
3706 3511
3706 3547
3706 3553
3706 3589
3706 3595
3706 3601
3706 3613
3706 3619
3706 3625
3706 3631
3706 3637
3706 3649
3706 3679
3706 3685
3706 3697
3712 3379
3712 3391
3712 3409
3712 3427
3712 3451
3712 3469
3712 3475
3712 3499
3712 3547
3712 3577
3712 3589
3712 3595
3712 3601
3712 3607
3712 3613
3712 3619
3712 3625
3712 3631
3712 3643
3712 3649
3712 3685
3712 3727
3712 3739
3718 3541
3718 3559
3718 3733
3724 2515
3724 3349
3

In [36]:
data_for_network.shape

(91135, 36)

In [37]:
from tensorflow.python.keras import backend as k
from keras.models import Sequential
from keras.layers import Dense,Activation
from keras.layers.recurrent import LSTM
import tensorflow
from sklearn import model_selection

Using TensorFlow backend.


In [40]:
data_count = data_for_network.shape[0]
data_rate = 0.8 * data_count
data_rate = int(data_rate)
X_train = data_for_network[:data_rate,:-1]
y_train = data_for_network[:data_rate, -1]
X_test = data_for_network[data_rate:,:-1]
y_test = data_for_network[data_rate:, -1]

print('X_train shape ',X_train.shape)
print('y_train shape ',y_train.shape)

X_train shape  (72908, 35)
y_train shape  (72908,)


In [43]:
 data_for_network.shape[0]
print("The radio of attack/all = ",sum(y_train)/y_train.shape[0])

The radio of attack/all =  0.3101854391836287


In [49]:
model = Sequential()
# model.add(LSTM(32, input_shape=(None,7),return_sequences=False))
model.add(Dense(128, input_shape=(35,), activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(1, activation="relu"))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=100)

Epoch 1/20
72908/72908 [==============================] - 1s 18us/step - loss: 4.9996 - acc: 0.6898
Epoch 2/20
72908/72908 [==============================] - 1s 15us/step - loss: 4.9996 - acc: 0.6898
Epoch 3/20
72908/72908 [==============================] - 1s 14us/step - loss: 4.9996 - acc: 0.6898
Epoch 4/20
72908/72908 [==============================] - 1s 14us/step - loss: 4.9996 - acc: 0.6898
Epoch 5/20
72908/72908 [==============================] - 1s 14us/step - loss: 4.9996 - acc: 0.6898
Epoch 6/20
72908/72908 [==============================] - 1s 15us/step - loss: 4.9996 - acc: 0.6898
Epoch 7/20
72908/72908 [==============================] - 1s 15us/step - loss: 4.9996 - acc: 0.6898
Epoch 8/20
72908/72908 [==============================] - 1s 15us/step - loss: 4.9996 - acc: 0.6898
Epoch 9/20
72908/72908 [==============================] - 1s 14us/step - loss: 4.9996 - acc: 0.6898
Epoch 10/20
72908/72908 [==============================] - 1s 15us/step - loss: 4.9996 - acc: 0.6898

In [48]:
model.evaluate(X_test, y_test, batch_size=64)

18227/18227 [==============================] - 0s 11us/step


[5.056414647862857, 0.686289570456754]